## File with AIMA functions

In file `search_mod.py` we have some functions implemented by AIMA. To run the code in this notebook, it is needed to run the following cell to import the functions from the file.


In [1]:
# Python scripts with the implementation of AIMA of the classes
# "Problem" and "Node" and the search algorithms.
from search import Problem, Node

## Reading the input


In [10]:
def read_input_file(filename):
    # Read the input file and return the list of lines.
    with open(filename, "r") as file:
        n_rows, n_cols = [int(x) for x in file.readline().split(" ")]

        # Read the map
        map = []

        for _ in range(n_rows):
            map.append([int(x) for x in file.readline().split(" ")])

        # Read the initial and final positions
        initial = tuple([int(x) for x in file.readline().split(" ")])
        goal = tuple([int(x) for x in file.readline().split(" ")])

        return map, initial, goal


read_input_file("exampleMap.txt")

([[3, 2, 4, 1], [2, 3, 1, 2], [1, 4, 2, 3]], (0, 3, 0), (1, 2, 8))

## Definition of the problem

We extend the class `Problem` implemented by AIMA to define the problem for this task, called: `MinningProblem`. To achieve this, we define the following methods:

- `__init__(self, initial, goal)`: This method initializes the problem with the initial state and the goal state.
- `is_valid_state(self, state)`: This method checks if a given state is valid, taking into account the number of rows and columns of the problem.
- `actions(self, state)`: This method returns the possible actions that can be taken from a given state. The actions will consists on the next states that can be reached from the current state.
- `result(self, state, action)`: This method returns the state that results from taking an action from a given state (in this case, the action is the next state).
- `h(self, node)`: This method returns the heuristic value of a given node.


In [37]:
class MinningProblem(Problem):
    """We have a board with rows and columns and a robot that can move in 8 directions."""

    def __init__(self, rows, columns, initial, goal):
        self.rows, self.columns = rows, columns
        self.initial, self.goal = initial, goal

    def is_valid_state(self, state):
        """
        Check that the given state is a possible
        state given a board with rows and columns
        """
        return all(x > 0 for x in state) and state[0] < self.rows and state[1] < self.columns and state[2] < 8

    def actions(self, state):
        """Return the possible movements of the robot given a state"""
        # Get the orientation of the robot
        orientation = state[2]
        # We always can rotate the robot 45 degrees to the left and to the right
        movements = [(state[0], state[1], (orientation + 1) % 8), (state[0], state[1], (orientation - 1) % 8)]

        # The startegy to move forward the robot is to move
        # it always and then check if new_state is valid
        # Calculate the new position on axis y
        new_state = list(state)
        if orientation >= 7 or orientation <= 1:
            new_state[0] = state[0] - 1
        elif orientation >= 3 and orientation <= 5:
            new_state[0] = state[0] + 1
        # Calculate the new position on axis x
        if orientation >= 1 and orientation <= 3:
            new_state[1] = state[1] + 1
        elif orientation >= 5 and orientation <= 7:
            new_state[1] = state[1] - 1

        if self.is_valid_state(new_state):
            movements.append(tuple(new_state))

        return movements

    def result(self, state, action):
        """Move the robot to the next state. The action consists in the next state"""
        return action

    def is_goal(self, state):
        """Check if the current state is the goal state"""
        return state[:1] == self.goal[:1]

    def h1(self, node):
        """
        Distance of Mahalanobis between the current state and the goal state.
        To elaborate this heuristic, we supose that all the positions of the board
        has cost 1 and the robot can rotate without cost. Also, we suppose that
        the robot only has to move forward in one axis to reach the goal state.
        """
        return max(abs(self.goal[0] - node.state[0]), abs(self.goal[1] - node.state[1]))

    def h2(self, node):
        """For this heurisit we suppose that all the positions of the board
        has cost 1.
        """
        if self.is_goal(node.state):
            return 0

        orientation = node.state[2]

        movements_y = self.goal[0] - node.state[0]
        movements_x = self.goal[1] - node.state[1]

        # Calculate the directions to move the robot to the goal state
        # in both axis
        orientation_y = 4 if movements_y > 0 else 0
        orientation_x = 2 if movements_x > 0 else 6

        # Calculate absolute values:
        movements_y = abs(movements_y)
        movements_x = abs(movements_x)

        # We have two options: rotate the robot to move diagonally towards
        # the goal state, or rotate the robot to move along the axis where
        # the robot is furthest from the goal state.
        if movements_y > movements_x:
            furthest_orientation = orientation_y
        else:
            furthest_orientation = orientation_x

        if orientation_y == 0 and orientation_x == 6:  # special case
            diagonal_orientation = 7
        else:
            diagonal_orientation = (orientation_y + orientation_x) // 2

        number_rotations_furthest = distance_between_orientations(orientation, furthest_orientation)
        number_rotations_diagonal = distance_between_orientations(orientation, diagonal_orientation)

        number_rotations = max(number_rotations_furthest, number_rotations_diagonal)
        diagonal_movements = min(movements_y, movements_x)
        straight_movements = abs(movements_y - movements_x)

        total_cost = number_rotations + diagonal_movements + straight_movements

        if number_rotations_furthest < number_rotations_diagonal:
            actions = (
                orientation,
                furthest_orientation,
                number_rotations_furthest,
                abs(movements_y - movements_x),
                diagonal_orientation,
                distance_between_orientations(furthest_orientation, diagonal_orientation),
                min(movements_y, movements_x),
            )
        else:
            actions = (
                orientation,
                diagonal_orientation,
                number_rotations_diagonal,
                min(movements_y, movements_x),
                furthest_orientation,
                distance_between_orientations(diagonal_orientation, furthest_orientation),
                abs(movements_y - movements_x),
            )

        print(
            f"Actions:\n- First, rotate robot from orientation {actions[0]} to {actions[1]} (cost: {actions[2]}).\n"
            + f"- Then move {actions[3]} steps with orientation {actions[1]} (cost: {actions[3]}).\n"
            + f"- Rotate robot from orientation {actions[1]} to {actions[4]} (cost: {actions[5]}).\n"
            + f"- Move {actions[6]} steps with orientation {actions[4]} (cost: {actions[6]}).\n"
            + f"- Total cost: {total_cost}."
        )

        return total_cost

    def h(self, node):
        return self.h2(node)


def distance_between_orientations(a, b):
    # Calculate the distance between two orientations
    return min((a - b) % 8, (b - a) % 8)

In [38]:
mp = MinningProblem(3, 4, (0, 3, 0), (2, 0, 8))

mp.actions((0, 3, 7))

[(0, 3, 0), (0, 3, 6)]

In [35]:
print(mp.h1(Node((0, 3, 0))))
print(mp.h2(Node((0, 3, 0))))

print(mp.h1(Node((0, 3, 2))))
print(mp.h2(Node((0, 3, 2))))

3
Actions:
- First, rotate robot from orientation 0 to 6 (cost: 2).
- Then move 1 steps with orientation 6 (cost: 1).
- Rotate robot from orientation 6 to 5 (cost: 1).
- Move 2 steps with orientation 5 (cost: 2).
- Total cost: 6.
6
3
Actions:
- First, rotate robot from orientation 2 to 5 (cost: 3).
- Then move 2 steps with orientation 5 (cost: 2).
- Rotate robot from orientation 5 to 6 (cost: 1).
- Move 1 steps with orientation 6 (cost: 1).
- Total cost: 7.
7


In [39]:
print(mp.h1(Node((2, 0, 2))))
print(mp.h2(Node((2, 0, 2))))

0
0
